In [ ]:
import pandas as pd
import numpy as np

In [ ]:
swap_rates = pd.read_csv("../data/market-data-swap-rates.csv")
# swap_rates.head(50)

In [ ]:
def ema(gp):
    #  Date is in the order natrually, so not necessary to rearrange the date
    N = len(gp)
    alpha = 2 / (N + 1)
    coe = []
    for i in range(N):
        coe.append(alpha * ((1 - alpha) ** i))
    coe.reverse()
    # print(coe)
    return sum(np.array(gp) * np.array(coe))


In [ ]:
# Sort the swap rates according to the Start date and the Tenor
gp_srs = swap_rates.groupby(['Start Date', 'Tenor'])['Swap Rate'].apply(ema).reset_index()
gp_srs

,Start Date,Tenor,Swap Rate
0,2022-01-13,10y,1.081333
1,2022-01-13,15y,1.265960
2,2022-01-13,18M,0.221625
3,2022-01-13,1y,0.195474
4,2022-01-13,2y,0.279370
...,...,...,...
28480,2039-01-12,2y,3.424505
28481,2039-01-12,3y,3.393758
28482,2039-01-12,4y,3.358897
28483,2039-01-12,5y,3.319531


In [ ]:
swap_vols = pd.read_csv("../data/market-data-swaption-vols.csv")
swap_vols["Date"].value_counts()

2022-03-09    810
2021-11-09    810
2023-10-30    810
2023-08-31    810
2021-07-12    810
             ... 
2023-04-25    405
2021-10-25    405
2022-05-23    405
2022-06-29    405
2022-02-02    405
Name: Date, Length: 754, dtype: int64

In [ ]:
# 按照strike和expiration分组，并计算平均波动率
# Sort the data according to the strike and the expiration and calculate the average vols as the final vols at that specific date.
gp_vols = swap_vols.groupby(['Expiry', 'Tenor', 'Date'])['Vols'].mean().reset_index()
gp_vols

,Expiry,Tenor,Date,Vols
0,10y,10y,2021-01-13,0.145596
1,10y,10y,2021-01-15,0.145449
2,10y,10y,2021-01-18,0.145109
3,10y,10y,2021-01-19,0.144115
4,10y,10y,2021-01-20,0.143333
...,...,...,...,...
61069,8y,8y,2024-01-08,0.256855
61070,8y,8y,2024-01-09,0.256540
61071,8y,8y,2024-01-10,0.256868
61072,8y,8y,2024-01-11,0.258929


In [ ]:
trade_information = pd.read_csv("../data/trade-information.csv")
trade_information.head(20)

,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound
0,dummyTrade1,USD: CMS:2Y,6M,5Y,0.0042,0.0379
1,dummyTrade2,USD: CMS:2Y,4M,5Y,0.0042,0.0379
2,dummyTrade3,USD: CMS:5Y,6M,5Y,0.0044,0.0397
3,dummyTrade4,USD: CMS:5Y,4M,5Y,0.0044,0.0397
4,dummyTrade5,USD: CMS:10Y,4M,5Y,0.0046,0.0417
5,dummyTrade6,USD: CMS:10Y,3M,5Y,0.0046,0.0417
6,dummyTrade7,USD: CMS:2Y,6M,10Y,0.0042,0.0379
7,dummyTrade8,USD: CMS:2Y,4M,10Y,0.0042,0.0379
8,dummyTrade9,USD: CMS:5Y,6M,10Y,0.0044,0.0397
9,dummyTrade10,USD: CMS:5Y,4M,10Y,0.0044,0.0397


In [ ]:
vegas = pd.read_csv("../data/trade-price-ir-vegas.csv")

# Data Combination of vegas and swap rates sheet and the volities sheet

In [ ]:
vegas.head(10)

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687
5,2022-09-02,dummyTrade1,USD,0,-215402.656152,1y,2023-09-04,10y,5.016274
6,2022-09-02,dummyTrade1,USD,5,-214640.494071,1y,2023-09-04,10y,3.458172
7,2022-09-02,dummyTrade1,USD,10,-213858.452424,1y,2023-09-04,10y,3.132444
8,2022-09-02,dummyTrade1,USD,25,-211345.804723,1y,2023-09-04,10y,12.184058
9,2022-09-02,dummyTrade1,USD,50,-206384.346001,1y,2023-09-04,10y,15.648409


In [ ]:
V_vegas = pd.merge(vegas, gp_vols, left_on = ["Value Date", "Expiry Bucket", "Tenor Bucket"], \
                right_on= ["Date", "Expiry", "Tenor"], how = "left")

V_vegas = V_vegas.drop(columns = ['Expiry', "Tenor", "Date"])
V_vegas

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,0.223827
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,0.223827
...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,0.267817
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,0.267817
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,0.267817
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,0.267817


In [ ]:
V_vegas2 = pd.merge(V_vegas, gp_srs, left_on = ["Value Date", "Tenor Bucket"], \
                right_on= ["Start Date", "Tenor"], how = "left")
V_vegas2.drop(columns= ["Tenor", "Start Date"])
V_vegas2

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols,Start Date,Tenor,Swap Rate
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827,2022-09-02,10y,1.554418
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827,2022-09-02,10y,1.554418
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827,2022-09-02,10y,1.554418
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,0.223827,2022-09-02,10y,1.554418
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,0.223827,2022-09-02,10y,1.554418
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,0.267817,2024-01-12,8y,2.044383
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,0.267817,2024-01-12,8y,2.044383
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,0.267817,2024-01-12,8y,2.044383
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,0.267817,2024-01-12,8y,2.044383


# Date: 2022-09-02; SwapRateDifference on 09-02;

In [ ]:
# V_vegas2.to_csv("Vegas_Vol_SwapRates.csv")

# Data Combination of vegas and dummy trade information

Read data：2 raw csv files

In [ ]:
# Read data files
CMS_data = pd.read_csv("../data/market-data-swap-rates.csv")
trade_info = pd.read_csv("../data/trade-information.csv")

Pay_frequency: number of payment per year

In [ ]:
# Feature Architecture: Pay Frequency
# Map the pay_frequency string to its corresponding numerical value
pay_frequency_mapping = {
    '6M': 2,
    '4M': 3,
    '3M': 4}
trade_info['pay_frequency'] = trade_info['pay_frequency'].map(pay_frequency_mapping)

Maturity: number of years

In [ ]:
# Feature Architecture: Maturity
# Map the maturity string to its corresponding numerical value
maturity_mapping = {
    '5Y': 5,
    '10Y': 10
}
trade_info['maturity'] = trade_info['maturity'].map(maturity_mapping)

Split market-data-swap-rate.csv: into 3 sub-files (according to underlying/tenor)

In [ ]:
# Create a mapping relationship from underlying to Tenor
underlying_to_tenor = {
    'USD: CMS:2Y': '2y',
    'USD: CMS:5Y': '5y',
    'USD: CMS:10Y': '10y'
}

# Multiply the values of upper_bound and lower_bound by 100 to standardize the scale for comparison
bound_info = trade_info.drop_duplicates('underlying').set_index('underlying') \
    [['upper_bound', 'lower_bound']].apply(lambda x: x * 100).to_dict('index')

# 根据underlying生成三个新的数据集
for underlying, tenor in underlying_to_tenor.items():
    filtered_cms_data = CMS_data[CMS_data['Tenor'] == tenor]
    underlying_swap_rate = f"swap-rates-{tenor}.csv"
    # filtered_cms_data.to_csv(underlying_swap_rate, index=False)
    # print(f"Saved {underlying_swap_rate}")

Saved swap-rates-2y.csv
Saved swap-rates-5y.csv
Saved swap-rates-10y.csv


Define function: transform into features;
underlying -> CMS mean/std/max/min;
           -> interest difference daily/weekly/monthly;
lower bound & upper bound -> proportion of time within Range (based on
                             comparison between raw CMS and bounds);

In [ ]:
# Feature Architecture: Underlying Assets
# Feature Architecture: Lower Bound & Upper Bound

def process_market_data(df, upper_bound, lower_bound):
# For Underlying Assets
# 1. Time series feature: STD of daily/weekly/monthly yield spreads
    # Convert date format and sort
    df['Start Date'] = pd.to_datetime(df['Start Date'])
    df = df.sort_values(by='Start Date')

    # Average of duplicate Start Dates
    df = df.groupby('Start Date').agg({'Swap Rate': 'mean'}).reset_index()

    # Fill missing dates and interpolate
    full_date_range = pd.date_range(start=df['Start Date'].min(), end=df['Start Date'].max(), freq='D')
    df = df.set_index('Start Date').reindex(full_date_range).rename_axis('Start Date').reset_index()
    df['Swap Rate'] = df['Swap Rate'].interpolate()

    # Calculate differences for different time intervals
    df['Daily Diff'] = df['Swap Rate'].diff()
    df['Weekly Diff'] = df['Swap Rate'].diff(7)
    df['Monthly Diff'] = df['Swap Rate'].diff(30)

    # Calculate the standard deviation of the differences
    daily_diff_std = df['Daily Diff'].std()
    weekly_diff_std = df['Weekly Diff'].std()
    monthly_diff_std = df['Monthly Diff'].std()

# 2. Statistical features: Average, STD, Maximum, Minimum of CMS
    # Calculate statistical features
    mean_cms = df['Swap Rate'].mean()
    std_cms = df['Swap Rate'].std()
    max_cms = df['Swap Rate'].max()
    min_cms = df['Swap Rate'].min()

# For Lower Bound & Upper Bound
    # Calculate the proportion of time within the interest rate range
    df['Within Range'] = (df['Swap Rate'] >= lower_bound) & (df['Swap Rate'] <= upper_bound)
    proportion_within_range = df['Within Range'].mean()

    return (daily_diff_std, weekly_diff_std, monthly_diff_std,
            mean_cms, std_cms, max_cms, min_cms,
            proportion_within_range)


Save processed feature data: into final_results.csv

In [ ]:
# Initialize an empty list to store all the features
final_results = []

# Process each sub-data file
for underlying, tenor in underlying_to_tenor.items():
    input_filename = f"swap-rates-{tenor}.csv"
    df = pd.read_csv(input_filename)

    # Retrieve upper_bound and lower_bound
    upper_bound = bound_info[underlying]['upper_bound']
    lower_bound = bound_info[underlying]['lower_bound']

    # Calculate STD of yield spreads, statistical features and time proportions
    daily_diff_std, weekly_diff_std, monthly_diff_std, \
        mean_cms, std_cms, max_cms, min_cms, \
        proportion_within_range \
        = process_market_data(df, upper_bound, lower_bound)

    # Append the results to the result list
    final_results.append(
        {'Underlying': underlying,
         'Daily_Diff_STD': daily_diff_std,
         'Weekly_Diff_STD': weekly_diff_std,
         'Monthly_Diff_STD': monthly_diff_std,
         'Mean_CMS': mean_cms,
         'STD_CMS': std_cms,
         'Max_CMS': max_cms,
         'Min_CMS': min_cms,
         'Proportion_Within_Range': proportion_within_range})

# Convert all results to a DataFrame
final_df = pd.DataFrame(final_results)

# Save the final DataFrame to a CSV file
# final_df.to_csv("final_results.csv", index=False)

Merge processed data with trade-information.csv: into final_feature_results.csv

In [ ]:
# Merge data into trade-information.csv
merged_df = pd.merge(trade_info, final_df, how='left', left_on='underlying', right_on='Underlying')

# Delete the 'Underlying' column
merged_df.drop(columns=['Underlying'], inplace=True)

# Delete the 'underlying' column
merged_df.drop(columns=['underlying'], inplace=True)

# Save the merged DataFrame to a CSV file
merged_df.to_csv("final_feature_results.csv", index=False)


Merge data: V_vegas2 and trade_featureResults -> final data in one csv.

In [ ]:
# Merge final_feature_results.csv into Vegas_Vol_SwapRates.csv
trade_featureResults = pd.read_csv('../codes/final_feature_results.csv')
merged_data = pd.merge(V_vegas2, trade_featureResults, how='left', left_on='Trade Name', right_on='trade name')

# Delete the 'trade name' column
merged_data.drop(columns=['trade name'], inplace=True)

# Check the results
print("Merged datasets results:")
merged_data.head(3)

Merged datasets results:


,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols,...,lower_bound,upper_bound,Daily_Diff_STD,Weekly_Diff_STD,Monthly_Diff_STD,Mean_CMS,STD_CMS,Max_CMS,Min_CMS,Proportion_Within_Range
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827,...,0.0042,0.0379,0.332305,0.194263,0.476413,2.451733,0.770452,4.239414,0.251352,0.966822
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827,...,0.0042,0.0379,0.332305,0.194263,0.476413,2.451733,0.770452,4.239414,0.251352,0.966822
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827,...,0.0042,0.0379,0.332305,0.194263,0.476413,2.451733,0.770452,4.239414,0.251352,0.966822


In [ ]:
merged_data.to_csv("Final_dataset.csv")

In [ ]:
merged_data.describe()

,Zero Rate Shock,TV,Vega,Vols,Swap Rate,pay_frequency,maturity,lower_bound,upper_bound,Daily_Diff_STD,Weekly_Diff_STD,Monthly_Diff_STD,Mean_CMS,STD_CMS,Max_CMS,Min_CMS,Proportion_Within_Range
count,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.895776e+06,3.919104e+06,3919104.0,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06
mean,6.250000e+00,-3.471395e+05,1.714511e+01,6.348165e-01,1.797211e+00,2.833333e+00,7.5,4.400000e-03,3.976667e-02,3.169565e-01,1.867454e-01,4.615927e-01,2.475673e+00,7.021760e-01,4.217337e+00,6.369079e-01,9.854512e-01
std,5.136167e+01,1.856180e+05,2.850144e+03,5.374183e-01,6.316145e-01,6.871844e-01,2.5,1.632993e-04,1.552060e-03,1.099498e-02,5.484974e-03,1.085112e-02,1.693220e-02,5.352307e-02,3.123304e-02,3.234081e-01,1.380856e-02
min,-1.000000e+02,-7.174312e+05,-1.385180e+06,3.110084e-02,3.504581e-01,2.000000e+00,5.0,4.200000e-03,3.790000e-02,3.071268e-01,1.813317e-01,4.507337e-01,2.451733e+00,6.397383e-01,4.173166e+00,2.513518e-01,9.668224e-01
25%,-1.375000e+01,-5.291890e+05,-1.342167e+00,2.496088e-01,1.333778e+00,2.000000e+00,5.0,4.200000e-03,3.790000e-02,3.071268e-01,1.813317e-01,4.507337e-01,2.451733e+00,6.397383e-01,4.173166e+00,2.513518e-01,9.668224e-01
50%,2.500000e+00,-2.242142e+05,1.336000e-04,4.464985e-01,1.791365e+00,3.000000e+00,7.5,4.400000e-03,3.970000e-02,3.114374e-01,1.846410e-01,4.576316e-01,2.487190e+00,6.963375e-01,4.239414e+00,6.166153e-01,9.896924e-01
75%,3.125000e+01,-1.857137e+05,7.386112e+00,9.132997e-01,2.208649e+00,3.000000e+00,10.0,4.600000e-03,4.170000e-02,3.323053e-01,1.942633e-01,4.764127e-01,2.488097e+00,7.704520e-01,4.239429e+00,1.042757e+00,9.998389e-01
max,1.000000e+02,-2.911344e+04,1.110263e+06,3.257591e+00,4.178719e+00,4.000000e+00,10.0,4.600000e-03,4.170000e-02,3.323053e-01,1.942633e-01,4.764127e-01,2.488097e+00,7.704520e-01,4.239429e+00,1.042757e+00,9.998389e-01
